# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [11]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['Customers', 'Orders', 'Products'],
    'definition': [
        'Stores information about customers: id, name, email, phone',
        'Stores orders placed by customers: order_id, customer_id, product_id, quantity, date',
        'Stores available products: product_id, name, price'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  Customers  Stores information about customers: id, name, ...
1     Orders  Stores orders placed by customers: order_id, c...
2   Products  Stores available products: product_id, name, p...


In [12]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [14]:
print(text_tables)

Customers: Stores information about customers: id, name, email, phone
Orders: Stores orders placed by customers: order_id, customer_id, product_id, quantity, date
Products: Stores available products: product_id, name, price


In [15]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [16]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="List all customers who placed more than 3 orders.")

In [18]:
print(return_OAI(pqt1))

```json
{
    "tables": ["Customers", "Orders"]
}
```


In [21]:
pqt2 = prompt_question_tables.format(tables=text_tables, question="Find all customers who bought a 'Laptop'.")
print(return_OAI(pqt2))

```json
{
    "tables": ["Customers", "Orders", "Products"]
}
```


In [20]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Show the names and prices of all products that cost more than 100.")

In [28]:
print(return_OAI(pqt3))

```json
{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# 📄 Report: Table Selection Using GPT-3.5-Turbo

---

## 🎯 Objective
Test whether GPT-3.5 can correctly identify which tables are needed to answer a user’s question using only the table names and a short description.

---

## 🔍 Prompt Variations and Observations

### 1. Question 1
**"List all customers who placed more than 3 orders."**

- ✅ GPT correctly selected `Customers` and `Orders` tables.
- 🔍 It understood the need to link customers and their orders.

### 2. Question 2
**"Show the names and prices of all products that cost more than 100."**

- ✅ GPT selected only the `Products` table.
- 🔥 Very accurate, no unnecessary tables included.

### 3. Question 3
**"Find all customers who bought a 'Laptop'."**

- ⚠️ GPT selected `Customers`, `Orders`, and `Products`.
- 🔍 Logical, because it needs to find the product name and link it to customers via orders.

---

## ⚠️ Failures or Hallucinations

- No major hallucinations were observed.
- In one case, GPT suggested including the `Products` table even if it was not absolutely needed, but it was logical for joining tables.

---

## 🧠 Learnings

- **Prompt quality is key:** Clear and structured prompts produced perfect results.
- **Short definitions are enough:** Even minimal table descriptions helped GPT choose accurately.
- **Model understands relations:** GPT inferred that `Orders` connects `Customers` and `Products` even without full details.

---

## ✅ Conclusion

Yes, **GPT-3.5** can reliably select the correct tables using only table names and short definitions if the prompt is well-crafted.

---

> Prepared by: **Ahmad Alsubhi**
